<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Text-Classification" data-toc-modified-id="Text-Classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Text Classification</a></span><ul class="toc-item"><li><span><a href="#Subjectivity-Dataset" data-toc-modified-id="Subjectivity-Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Subjectivity Dataset</a></span></li><li><span><a href="#Tokenization" data-toc-modified-id="Tokenization-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Tokenization</a></span><ul class="toc-item"><li><span><a href="#Simple-Tokenization" data-toc-modified-id="Simple-Tokenization-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simple Tokenization</a></span></li><li><span><a href="#Much-better-tokenization-with-Spacy" data-toc-modified-id="Much-better-tokenization-with-Spacy-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Much better tokenization with Spacy</a></span></li></ul></li><li><span><a href="#Split-dataset-in-train-and-validation" data-toc-modified-id="Split-dataset-in-train-and-validation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Split dataset in train and validation</a></span></li><li><span><a href="#Word-to-index-mapping" data-toc-modified-id="Word-to-index-mapping-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Word to index mapping</a></span></li><li><span><a href="#Sentence-encoding" data-toc-modified-id="Sentence-encoding-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Sentence encoding</a></span></li><li><span><a href="#Embedding-layer" data-toc-modified-id="Embedding-layer-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Embedding layer</a></span></li><li><span><a href="#Continuous-Bag-of-Words-Model" data-toc-modified-id="Continuous-Bag-of-Words-Model-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Continuous Bag of Words Model</a></span></li></ul></li><li><span><a href="#Training-the-CBOW-model" data-toc-modified-id="Training-the-CBOW-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training the CBOW model</a></span></li><li><span><a href="#Data-loaders-for-SGD" data-toc-modified-id="Data-loaders-for-SGD-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data loaders for SGD</a></span></li></ul></div>

In [1]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Text Classification
In this part of the tutorial we develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [2]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [3]:
#unpack_dataset()

In [4]:
!ls data

plot.tok.gt9.5000   quote.tok.gt9.5000  subjdata.README.1.0


In [5]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [6]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [7]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [8]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [9]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [10]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Much better tokenization with Spacy

In [ ]:
#!pip install -U spacy

In [ ]:
import spacy

In [ ]:
# first time run this
#!python3 -m spacy download en

In [ ]:
tok = spacy.load('en')

In [ ]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [ ]:
len(obj_lines)

In [ ]:
obj_lines[0]

In [ ]:
test = tok(obj_lines[0])

In [ ]:
np.array([x for x in test])

## Split dataset in train and validation

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [13]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'), array([1., 0., 0., 1., 1.]))

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [16]:
from collections import defaultdict

In [17]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [18]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [19]:
#word_count

In [20]:
len(word_count.keys())

21415

In [21]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [22]:
len(word_count.keys())

4065

In [23]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [24]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [25]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_val_len = np.array([len(x.split()) for x in X_val])

In [26]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

43.0

In [27]:
X_train[0]

'will god let her fall or give her a new path ?'

In [28]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

8

In [29]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([11,  3,  6,  7,  2, 12,  9,  7, 10,  4,  5,  8])

In [30]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [31]:
encode_sentence(X_train[0])

array([11,  3,  6,  7,  2, 12,  9,  7, 10,  4,  5,  8,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [32]:
x_train_len = np.minimum(x_train_len, 40)
x_val_len = np.minimum(x_val_len, 40)

In [33]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [34]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [35]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.9722,  0.9138,  0.0743, -0.1021],
        [-0.0091, -0.4712,  1.2977, -1.2585],
        [ 0.1368,  1.4354, -0.0935,  0.1110],
        [ 0.7230, -0.9195,  0.9880,  1.2590],
        [-0.4534,  0.8292, -0.2036,  0.3768],
        [ 1.1694,  1.6533,  1.1898,  1.0617],
        [-1.1252, -0.2761, -0.1112,  0.7598],
        [ 0.9987,  1.0012, -0.3599,  0.5257],
        [ 1.2248, -0.2419,  0.2870, -1.5904]])

Note that the `padding_idx` has embedding vector 0.

In [36]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[-0.9722,  0.9138,  0.0743, -0.1021],
         [ 0.7230, -0.9195,  0.9880,  1.2590],
         [-0.9722,  0.9138,  0.0743, -0.1021],
         [-0.4534,  0.8292, -0.2036,  0.3768],
         [-0.9722,  0.9138,  0.0743, -0.1021],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [37]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [38]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [39]:
embed(a)

tensor([[[-0.9722,  0.9138,  0.0743, -0.1021],
         [ 0.7230, -0.9195,  0.9880,  1.2590],
         [-0.9722,  0.9138,  0.0743, -0.1021]],

        [[-0.9722,  0.9138,  0.0743, -0.1021],
         [ 0.1368,  1.4354, -0.0935,  0.1110],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]])

In [40]:
embed(a).sum(dim=1)

tensor([[-1.2213,  0.9080,  1.1367,  1.0548],
        [-0.8354,  2.3491, -0.0192,  0.0089]])

In [41]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[-0.4071,  0.3027,  0.3789,  0.3516],
        [-0.4177,  1.1746, -0.0096,  0.0044]])

## Continuous Bag of Words Model

In [42]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

In [43]:
model = CBOW(vocab_size=5, emb_size=3)

In [44]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 1.6292,  1.2889,  0.7647],
        [ 2.5952, -0.9427,  0.3432],
        [ 0.5775, -2.7160, -1.4606],
        [ 1.2119,  0.8058, -0.0705]])

In [45]:
s = s.view(s.shape[0], 1)
model(a, s)

tensor([[ 0.1384],
        [ 0.5663]])

# Training the CBOW model 

In [51]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [52]:
def val_metrics(model):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    s = torch.Tensor(x_val_len).view(x_val_len.shape[0], 1)
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [53]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.6892560720443726, 0.5245000123977661)

In [54]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        s = torch.Tensor(x_train_len).view(x_train_len.shape[0], 1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, val_accuracy = val_metrics(model)
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (loss.item(), val_loss, val_accuracy))

In [55]:
train_epocs(model, epochs=10, lr=0.1)

train_loss 0.689 val_loss 0.651 val_accuracy 0.582
train_loss 0.639 val_loss 0.559 val_accuracy 0.766
train_loss 0.544 val_loss 0.467 val_accuracy 0.825
train_loss 0.440 val_loss 0.375 val_accuracy 0.866
train_loss 0.337 val_loss 0.321 val_accuracy 0.867
train_loss 0.270 val_loss 0.274 val_accuracy 0.893
train_loss 0.213 val_loss 0.258 val_accuracy 0.900
train_loss 0.183 val_loss 0.253 val_accuracy 0.902
train_loss 0.160 val_loss 0.250 val_accuracy 0.909
train_loss 0.135 val_loss 0.260 val_accuracy 0.905


In [56]:
train_epocs(model, epochs=10, lr=0.01)

train_loss 0.120 val_loss 0.257 val_accuracy 0.907
train_loss 0.115 val_loss 0.257 val_accuracy 0.909
train_loss 0.113 val_loss 0.256 val_accuracy 0.907
train_loss 0.110 val_loss 0.256 val_accuracy 0.908
train_loss 0.107 val_loss 0.255 val_accuracy 0.908
train_loss 0.103 val_loss 0.255 val_accuracy 0.907
train_loss 0.100 val_loss 0.255 val_accuracy 0.906
train_loss 0.098 val_loss 0.256 val_accuracy 0.908
train_loss 0.095 val_loss 0.257 val_accuracy 0.906
train_loss 0.092 val_loss 0.257 val_accuracy 0.906


# Data loaders for SGD

Nearly all of deep learning is powered by one very important algorithm: **stochastic gradient descent (SGD)**. SGD can be seeing as an approximation of **gradient descent** (GD). In GD you have to run through *all* the samples in your training set to do a single itaration. In SGD you use *only one* or *a subset*  of training samples to do the update for a parameter in a particular iteration. The subset use in every iteration is called a **batch** or **minibatch**.

In [57]:
from torch.utils.data import Dataset, DataLoader

Next we are going to create a data loader. The data loader provides the following features:
* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

In [58]:
def encode_sentence2(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [59]:
encode_sentence2(X_train[0])

(array([11,  3,  6,  7,  2, 12,  9,  7, 10,  4,  5,  8,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32), 12)

In [60]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence2(x)
        return x, self.y[idx], s
    
sub_dataset_train = SubjectivityDataset(X_train, y_train)

In [61]:
train_loader = DataLoader(sub_dataset_train, batch_size=5, shuffle=True)
x, y, s = next(iter(train_loader))

In [62]:
x, y, s

(tensor([[  243,  2146,     1,   384,    57,     1,    57,     1,     1,
             37,   559,     1,     1,  2632,     1,    42,    24,    15,
            645,  3014,  2936,    88,     1,    37,     1,  2632,  2029,
              1,    80,     1,    23,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [  147,    15,  1075,  1910,   362,    42,  3155,   125,   588,
             32,   588,    63,    40,    41,  1479,    57,  2537,    24,
             15,     1,    57,  1725,   152,    40,   233,    23,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0],
         [   15,  1873,    24,   119,    29,  2288,   477,  1575,    24,
           1645,    42,    24,    15,   127,   635,   436,   147,   173,
            128,  1551,   129,   436,    72,   704,     1,    42,   977,
             24,  2144,    42,    29,   738,     1,   434,   104,    23,
              0,     0,     0,     0],
       

In [63]:
model = CBOW(vocab_size=V, emb_size=50)

In [64]:
train_loader = DataLoader(sub_dataset_train, batch_size=500, shuffle=True)

In [ ]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        total_loss = 0
        total = 0
        model.train()
        for x, y, s in train_loader:
            x = x.type(torch.LongTensor)  #.cuda()
            y = y.type(torch.FloatTensor).unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x, s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
            train_loss
        val_loss, val_accuracy = val_metrics(model)
        
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (loss.item(), val_loss, val_accuracy))

In [ ]:
train_epocs(model, epochs=10)